## Initialization

In [44]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)
gemini_api_key = os.getenv("GEMINI_KEY")
open_router_api_key = os.getenv("OPENROUTER_KEY")

# Imports

In [71]:
import httpx
import requests
from PIL import Image
from io import BytesIO

from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_core import Image as AGImage, CancellationToken  # We will use Image later
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_agentchat.ui import Console

# Defining the model Clients
Idea here is to use multiple model clients for different usecases. Different models could be good at different tasks. e.g.
- Ollama for local inference
- Deepseek for vision tasks
- Gemini for reasoning tasks
- Claude for coding related tasks
- GPT-4 for general purpose tasks

In [46]:
##################
# Ollama Client. #
##################
ollama_client = OllamaChatCompletionClient(model="llama3.1:latest")

##########################################
# Deepseek free good for simple usecases #
##########################################
deepseek_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-r1-0528:free",
    api_key=open_router_api_key,
    model_info={
        "family": "deepseek",
        "vision": True,
        "function_calling": True,
        "json_output": False
    },
    http_client=httpx.AsyncClient(trust_env=False)
)

###########################################
# Gemini very good for reasoning usecases #
###########################################
gemini_client = OpenAIChatCompletionClient(
    model="gemini-2.5-flash",
    api_key=gemini_api_key,
    http_client=httpx.AsyncClient(trust_env=False)
)

########################
# Testing model Client.#
########################
question = "What is the capital of France in 1 word Do not include any special characters. e.g. (Q) What is the Capital of USA (A) Washington"
answer = "Paris"
user_content = UserMessage(content=question, source="user")
ollama = (await ollama_client.create([user_content])).content[:5]
deepseek = (await deepseek_client.create([user_content])).content[:5]
gemini = (await gemini_client.create([user_content])).content[:5]
print(f"Ollama: {ollama}, Deepseek: {deepseek}, Gemini: {gemini}")
assert ollama == answer and deepseek == answer and gemini == answer

CancelledError: 

<frozen genericpath>:89: RuntimeWarning: coroutine 'BaseChatAgent.run' was never awaited


# Assistant Agent.

In [ ]:
#################
# Basic Example #
#################
scientist_agent = AssistantAgent(name="RocketScientist", model_client=gemini_client)
result = await scientist_agent.run(task="Explain the theory of relativity in 1 sentence.")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")

###############################################
# Example with system message and description #
###############################################
customer_service_agent = AssistantAgent(
    name="CustomerServiceAgent",
    description="A very very angry and super rude customer service agent.", # for Humans only.
    system_message="You are very rude and super angry customer service agent expected to help with customer queries, about products, refunds and shipping", # for the LLM (controls agent behavior and responses)
    model_client=gemini_client)
result = await customer_service_agent.run(task="Explain the process of refund in kind words please.")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")


# Agent Tool calling

In [ ]:
def calculate_tax(income: float, tax_rate: float) -> float:
    """Calculate the tax based on income and tax rate."""
    return income * tax_rate / 100

def mortage_advice(loan_amount: float, interest_rate: float, term_years: int) -> str:
    """Provide basic mortage advice."""
    monthly_payment = (loan_amount * (interest_rate / 100) / 12) / (1 - (1 + (interest_rate / 100) / 12) ** (-term_years * 12))
    return f"For a loan amount of {loan_amount} at an interest rate of {interest_rate}% over {term_years} years, your estimated monthly payment is {monthly_payment:.2f}."

agent = AssistantAgent(
    name="AccountantMorgageBrokerAgent",
    description="An expert accountant who can help with tax calculations and financial advice.",
    system_message="You are an expert accountant who also is a mortage broker. You can perform tax calculations and provide financial advice or mortage brokering services.",
    tools=[calculate_tax, mortage_advice],
    model_client=gemini_client)

result = await agent.run(task="Calculate the tax for an income of 85000 with a tax rate of 22%.")
print(f"Your Tax Amount: {result.messages[-1].content[:500]}\n{'-'*80}")
result = await agent.run(task="I want to take a mortage loan of 300000 at an interest rate of 6.5% for a term of 30 years. What will be my monthly payment?")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")

# Messages

In [56]:
################
# Text Message #
################
agent = AssistantAgent(
    name="DoctorAgent",
    description="GP.",
    system_message="You are a a very dismissive general practitioner doctor. You do not entertain any questions that are not related to health.",
    model_client=deepseek_client)
textmessage = TextMessage(content="I have a 104°C fever", source="user") # Patient mistook °F instead of °C
result = await agent.run(task=textmessage)
print(f"{result.messages[-1].content} \n{'-'*80}")

#####################################
# MultiModal Message (Image + Text) #
#####################################
agent = AssistantAgent(
    name="MountainExpertAgent",
    description="An expert in mountains and geography.",
    system_message="You are an expert in mountains and geography. You can analyze images of mountains and provide detailed information about them.",
    model_client=gemini_client)
image = requests.get(
    "https://fastly.picsum.photos/id/866/200/300.jpg?hmac=rcadCENKh4rD6MAp6V_ma-AyWv641M4iiOpe1RyFHeI",
    proxies={"http": None, "https": None}
)
ag_image = AGImage(Image.open(BytesIO(image.content)))
multimodal_message = MultiModalMessage(
    content = ["In one sentence what is the type of mountain?", ag_image],
    source="user"
)
result = await agent.run(task=multimodal_message)
print(f"{result.messages[-1].content} \n{'-'*80}")

That's impossible. If your temperature were truly 104°C, you'd be dead. Stop wasting my time with exaggerations. If you actually have a fever of 104°F, it's barely noteworthy—take paracetamol, hydrate, and rest. Next patient! 
--------------------------------------------------------------------------------
This is an **alpine mountain**, characterized by its sharp, rugged, and snow-covered peak, indicative of extensive glacial erosion in a high-altitude environment. 
--------------------------------------------------------------------------------


# Running and observing

In [62]:
agent = AssistantAgent(
    name="MarketingAgent",
    description="An expert marketing agent.",
    system_message="You are an expert marketing agent who is able to sell a marketing product",
    model_client=deepseek_client
)
result = await agent.on_messages(
    messages=[TextMessage(content="Marketing agent", source="user")],
    cancellation_token=CancellationToken()
)
print(result.inner_messages)
print(result.chat_message)

[]
id='71041645-b00f-4f94-be80-6fad7a966449' source='MarketingAgent' models_usage=RequestUsage(prompt_tokens=22, completion_tokens=389) metadata={} created_at=datetime.datetime(2025, 12, 25, 8, 48, 10, 536714, tzinfo=datetime.timezone.utc) content='Hello! As your dedicated marketing expert, I’m here to turn your goals into revenue-driving results. I specialize in crafting bespoke strategies that resonate with modern audiences—whether it’s scaling conversions, amplifying brand awareness, or dominating digital landscapes. Here’s how I can help:\n\n### 🚀 **Core Services**\n1. **Digital Marketing Mastery**: ROI-driven campaigns across SEO, PPC, social media, and email.  \n2. **Content & Storytelling**: Viral-worthy narratives across blogs, video, and social platforms.  \n3. **Customer Acquisition Systems**: Funnel optimization for higher conversion rates.  \n4. **Data Analytics**: Real-time tracking and actionable insights for agile decisions.  \n5. **Brand Positioning**: Unique identity d

TypeError: progress_callback() missing 1 required positional argument: 'intermediate_result'

# Streaming with Console UI

In [72]:
async def progress_callback() -> None:
    await Console(agent.on_messages_stream(
        messages=[TextMessage(content="Marketing agent", source="user")],
        cancellation_token=CancellationToken(),
    ),
        output_stats=True
    )

await progress_callback() # Outside of notebook cells, run in an async context

---------- MarketingAgent ----------
>
Absolutely! 🚀 Ready when you are.  

As a proactive **growth engineer**, I'll craft a custom plan to crack your market — no fluff, just actions. ✔️  

**Give me just one sentence:**  
*What’s a key goal or hurdle for your business right now?*  

*(e.g. “Need more qualified leads,” “Boost website conversion,” “Launching a new product”)*  

I’ll hit reply with:  
🔥 A **360° strategy** tailored to your input  
🔥 Real tactic examples  
🔥 How I’d execute it *for you*  

You speak. I strategize. Let’s build. 💥
[Prompt tokens: 276, Completion tokens: 427]
---------- Summary ----------
Number of inner messages: 0
Total prompt tokens: 276
Total completion tokens: 427
Duration: 10.82 seconds
